## Dataset Overview


- **Título**: Marketing Bancário
- **Criado por**: Paulo Cortez (Univ. Minho) e Sérgio Moro (ISCTE-IUL) @ 2012
- **Total de Instâncias**: 45.211 (bank-full.csv)
- **Número de Atributos**: 16 variáveis de entrada + 1 variável alvo
- **Valores Ausentes**: Nenhum

### Contexto

Este conjunto de dados está relacionado a campanhas de marketing direto de uma instituição bancária portuguesa. As campanhas foram realizadas por meio de ligações telefônicas, muitas vezes exigindo múltiplos contatos com o mesmo cliente para determinar se ele iria subscrever um depósito a prazo.

### Variável Alvo

- **y**: Se o cliente subscreveu um depósito a prazo (binário: "yes"/"no")

### Descrição das Variáveis

#### Dados Pessoais do Cliente

1. **age**

   - Tipo: Numérico
   - Descrição: Idade do cliente

2. **job**

   - Tipo: Categórico
   - Valores: "admin.", "unknown", "unemployed", "management", "housemaid", "entrepreneur", "student", "blue-collar", "self-employed", "retired", "technician", "services"
   - Descrição: Tipo de emprego do cliente

3. **marital**

   - Tipo: Categórico
   - Valores: "married", "divorced", "single"
   - Observação: "divorced" inclui tanto divorciados quanto viúvos

4. **education**

   - Tipo: Categórico
   - Valores: "unknown", "secondary", "primary", "tertiary"
   - Descrição: Nível de escolaridade do cliente

5. **default**

   - Tipo: Binário
   - Valores: "yes", "no"
   - Descrição: Se o cliente possui crédito em inadimplência

6. **balance**

   - Tipo: Numérico
   - Descrição: Saldo médio anual em euros

7. **housing**

   - Tipo: Binário
   - Valores: "yes", "no"
   - Descrição: Se o cliente possui empréstimo habitacional

8. **loan**
   - Tipo: Binário
   - Valores: "yes", "no"
   - Descrição: Se o cliente possui empréstimo pessoal

#### Informações do Contato da Campanha Atual

9. **contact**

   - Tipo: Categórico
   - Valores: "unknown", "telephone", "cellular"
   - Descrição: Tipo de comunicação utilizada para o contato

10. **day**

    - Tipo: Numérico
    - Descrição: Último dia de contato no mês

11. **month**

    - Tipo: Categórico
    - Valores: "jan", "feb", "mar", ..., "nov", "dec"
    - Descrição: Último mês de contato no ano

12. **duration**
    - Tipo: Numérico
    - Descrição: Duração do último contato em segundos

#### Informações da Campanha

13. **campaign**

    - Tipo: Numérico
    - Descrição: Número de contatos realizados durante esta campanha para este cliente (inclui o último contato)

14. **pdays**

    - Tipo: Numérico
    - Descrição: Número de dias desde o último contato do cliente em uma campanha anterior
    - Observação: -1 significa que o cliente não foi previamente contatado

15. **previous**

    - Tipo: Numérico
    - Descrição: Número de contatos realizados antes desta campanha para este cliente

16. **poutcome**
    - Tipo: Categórico
    - Valores: "unknown", "other", "failure", "success"
    - Descrição: Resultado da campanha de marketing anterior


## Exploração


In [1]:
import pandas as pd
import numpy as np
import sklearn as skl
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from imblearn.over_sampling import SMOTE

In [ ]:
df = pd.read_csv("datasets/bank.csv", sep=";")
df[df['y']=='yes'].head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
13,20,student,single,secondary,no,502,no,no,cellular,30,apr,261,1,-1,0,unknown,yes
30,68,retired,divorced,secondary,no,4189,no,no,telephone,14,jul,897,2,-1,0,unknown,yes
33,32,management,single,tertiary,no,2536,yes,no,cellular,26,aug,958,6,-1,0,unknown,yes
34,49,technician,married,tertiary,no,1235,no,no,cellular,13,aug,354,3,-1,0,unknown,yes
36,78,retired,divorced,primary,no,229,no,no,telephone,22,oct,97,1,-1,0,unknown,yes


In [ ]:
df[df['y']=='no'].head()

In [3]:
feature_cols = [
  "job",
  "marital",
  "education",
  "contact",
  "housing",
  "loan",
  "default",
  "day",
]

In [4]:
df.loc[df['y'] == 'yes', feature_cols].iloc[30].to_dict()

{'job': 'blue-collar',
 'marital': 'single',
 'education': 'secondary',
 'contact': 'cellular',
 'housing': 'no',
 'loan': 'yes',
 'default': 'no',
 'day': 10}

In [5]:
def ingest_and_prep_data(
    bank_dataset: str = "datasets/bank-full.csv",
) -> tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame, OneHotEncoder]:
    
    df = pd.read_csv(bank_dataset, delimiter=";", decimal=",")

    feature_cols = [
        "job",
        "marital",
        "education",
        "contact",
        "housing",
        "loan",
        "default",
        "day",
    ]

    X = df[feature_cols].copy()
    y = df["y"].apply(lambda x: 1 if x == "yes" else 0).copy()

    print(f"Dataset shape: {X.shape}, {y.shape}")

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=42
    )

    # Feature engineering
    enc = OneHotEncoder(handle_unknown="ignore")
    X_train_enc = enc.fit_transform(X_train)
    X_test_enc = enc.transform(X_test)

    return X_train_enc, X_test_enc, y_train, y_test, enc

In [6]:
def rebalance_classes(
    X: pd.DataFrame, y: pd.DataFrame
) -> tuple[pd.DataFrame, pd.DataFrame]:
    sm = SMOTE()
    X_balanced, y_balanced = sm.fit_resample(X, y)
    return X_balanced, y_balanced

In [7]:
def get_hyperparam_grid() -> dict:
    # Hyperparameter optimisation
    n_estimators = [int(x) for x in np.linspace(start=200, stop=2000, num=10)]
    # Number of features to consider at every split
    max_features = [
        "log2",
        "sqrt",
    ]  # ['auto', 'sqrt'] #TODO: auto throws some errors, remove from book example?
    # Maximum number of levels in tree
    max_depth = [int(x) for x in np.linspace(10, 110, num=11)]
    max_depth.append(None)
    # Minimum number of samples required to split a node
    min_samples_split = [2, 5, 10]
    # Minimum number of samples required at each leaf node
    min_samples_leaf = [1, 2, 4]
    # Method of selecting samples for training each tree
    bootstrap = [True, False]  # Create the random grid
    random_grid = {
        "n_estimators": n_estimators,
        "max_features": max_features,
        "max_depth": max_depth,
        "min_samples_split": min_samples_split,
        "min_samples_leaf": min_samples_leaf,
        "bootstrap": bootstrap,
    }
    return random_grid

In [8]:
def get_randomised_rf_cv(
    random_grid: dict,
) -> skl.model_selection._search.RandomizedSearchCV:
    # Use the random grid to search for best hyperparameters
    # First create the base model to tune
    rf = RandomForestClassifier()
    # Random search of parameters, using 3 fold cross validation,
    # search across 3 different combinations, and use all available cores
    rf_random = RandomizedSearchCV(
        estimator=rf,
        param_distributions=random_grid,
        n_iter=3,
        cv=3,
        verbose=2,
        random_state=42,
        n_jobs=-1,
        scoring="f1",
    )
    return rf_random

## Treinamento do modelo


In [9]:
import mlflow
from mlflow import MlflowClient

In [10]:
client = MlflowClient()

In [11]:
mlflow.set_tracking_uri(uri="http://localhost:8080")

In [12]:
import joblib

def main():
    with mlflow.start_run() as run:
        X_train, X_test, y_train, y_test, enc = ingest_and_prep_data()

        X_balanced, y_balanced = rebalance_classes(X_train, y_train)

        rf_random = get_randomised_rf_cv(random_grid=get_hyperparam_grid())

        mlflow.log_params(rf_random.get_params())
        rf_random.fit(X_balanced, y_balanced)

        # X_test já está codificado, não precisa transformar novamente
        y_pred = rf_random.predict(X_test)

        mlflow.log_metrics({"f1": f1_score(y_test, y_pred)})

        # Save encoder as part of the model artifacts
        encoder_path = "encoder.joblib"
        joblib.dump(enc, encoder_path)
        mlflow.log_artifact(encoder_path)

        # Log the model with the encoder path in the context
        mlflow.sklearn.log_model(
            sk_model=rf_random,
            name="rf-model",
            input_example=X_test,
            registered_model_name="random-forest",
            tags={"version": "latest"},
        )

        model_uri = f"runs:/{run.info.run_id}/rf-model"
        mv = mlflow.register_model(
            model_uri, "random-forest", tags={"version": "latest"}
        )

        print(f"Name: {mv.name}")
        print(f"Version: {mv.version}")


if __name__ == "__main__":
    main()

Dataset shape: (45211, 8), (45211,)
Fitting 3 folds for each of 3 candidates, totalling 9 fits
[CV] END bootstrap=True, max_depth=50, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=200; total time=  36.2s
[CV] END bootstrap=True, max_depth=50, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=200; total time=  38.1s
[CV] END bootstrap=True, max_depth=50, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=200; total time=  38.3s
[CV] END bootstrap=False, max_depth=90, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=600; total time= 1.6min
[CV] END bootstrap=False, max_depth=90, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=600; total time= 1.7min
[CV] END bootstrap=False, max_depth=90, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=600; total time= 1.7min
[CV] END bootstrap=False, max_depth=60, max_features=log2, min_samples_leaf=2, m

2025/11/06 12:13:47 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
Registered model 'random-forest' already exists. Creating a new version of this model...
2025/11/06 12:15:35 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: random-forest, version 3
Created version '3' of model 'random-forest'.
Registered model 'random-forest' already exists. Creating a new version of this model...
2025/11/06 12:15:35 WARNING mlflow.tracking._model_registry.fluent: Run with id 8bcfcdb44e6542eda2106b8d4cc3485f has no artifacts at artifact path 'rf-model', registering model based on models:/m-ad55203cf9124f75add0b92087fcd573 instead
2025/11/06 12:15:35 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: random-forest, version 4
Created version '4' of

Name: random-forest
Version: 4
🏃 View run beautiful-steed-940 at: http://localhost:8080/#/experiments/0/runs/8bcfcdb44e6542eda2106b8d4cc3485f
🧪 View experiment at: http://localhost:8080/#/experiments/0
